In [1]:
import findspark
findspark.init("/usr/local/spark")

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Pyspark Linear Regression").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import *

In [5]:
data=spark.read.load("linregdata1.csv",format='csv',sep=',',inferSchema='true',header='true')

In [6]:
data.printSchema()

root
 |-- temperature: double (nullable = true)
 |-- exhaust_vacuum: double (nullable = true)
 |-- ambient_pressure: double (nullable = true)
 |-- relative_humidity: double (nullable = true)
 |-- energy_output: double (nullable = true)



In [7]:
data.describe()

DataFrame[summary: string, temperature: string, exhaust_vacuum: string, ambient_pressure: string, relative_humidity: string, energy_output: string]

In [8]:
features=['temperature','exhaust_vacuum','ambient_pressure','relative_humidity']

In [9]:
lr_data=data.select(col("energy_output").alias("label"),*features)

In [10]:
lr_data.printSchema()

root
 |-- label: double (nullable = true)
 |-- temperature: double (nullable = true)
 |-- exhaust_vacuum: double (nullable = true)
 |-- ambient_pressure: double (nullable = true)
 |-- relative_humidity: double (nullable = true)



In [11]:
lr_data.show()

+------+-----------+--------------+----------------+-----------------+
| label|temperature|exhaust_vacuum|ambient_pressure|relative_humidity|
+------+-----------+--------------+----------------+-----------------+
|480.48|       8.34|         40.77|         1010.84|            90.01|
|445.75|      23.64|         58.49|          1011.4|             74.2|
|438.76|      29.74|          56.9|         1007.15|            41.91|
|453.09|      19.07|         49.69|         1007.22|            76.79|
|464.43|       11.8|         40.66|         1017.13|             97.2|
|470.96|      13.97|         39.16|         1016.05|             84.6|
|442.35|       22.1|         71.29|          1008.2|            75.38|
| 464.0|      14.47|         41.76|         1021.98|            78.41|
|428.77|      31.25|         69.51|         1010.25|            36.83|
|484.31|       6.77|         38.18|          1017.8|            81.13|
|435.29|      28.28|         68.67|         1006.36|             69.9|
|451.4

In [15]:
#vectorization
va=VectorAssembler(inputCols=features, outputCol="unscaled_features")
va_data=va.transform(lr_data)
va_data.show()

+------+-----------+--------------+----------------+-----------------+--------------------+
| label|temperature|exhaust_vacuum|ambient_pressure|relative_humidity|   unscaled_features|
+------+-----------+--------------+----------------+-----------------+--------------------+
|480.48|       8.34|         40.77|         1010.84|            90.01|[8.34,40.77,1010....|
|445.75|      23.64|         58.49|          1011.4|             74.2|[23.64,58.49,1011...|
|438.76|      29.74|          56.9|         1007.15|            41.91|[29.74,56.9,1007....|
|453.09|      19.07|         49.69|         1007.22|            76.79|[19.07,49.69,1007...|
|464.43|       11.8|         40.66|         1017.13|             97.2|[11.8,40.66,1017....|
|470.96|      13.97|         39.16|         1016.05|             84.6|[13.97,39.16,1016...|
|442.35|       22.1|         71.29|          1008.2|            75.38|[22.1,71.29,1008....|
| 464.0|      14.47|         41.76|         1021.98|            78.41|[14.47,41.

In [16]:
ss = StandardScaler(inputCol="unscaled_features",outputCol="features")
ss_model = ss.fit(va_data)
ss_data=ss_model.transform(va_data)
ss_data.show()

+------+-----------+--------------+----------------+-----------------+--------------------+--------------------+
| label|temperature|exhaust_vacuum|ambient_pressure|relative_humidity|   unscaled_features|            features|
+------+-----------+--------------+----------------+-----------------+--------------------+--------------------+
|480.48|       8.34|         40.77|         1010.84|            90.01|[8.34,40.77,1010....|[1.11909157444034...|
|445.75|      23.64|         58.49|          1011.4|             74.2|[23.64,58.49,1011...|[3.17210129733451...|
|438.76|      29.74|          56.9|         1007.15|            41.91|[29.74,56.9,1007....|[3.99062151365179...|
|453.09|      19.07|         49.69|         1007.22|            76.79|[19.07,49.69,1007...|[2.55888205330664...|
|464.43|       11.8|         40.66|         1017.13|             97.2|[11.8,40.66,1017....|[1.58336697582687...|
|470.96|      13.97|         39.16|         1016.05|             84.6|[13.97,39.16,1016...|[1.87

In [18]:
#split data into training and test set
(training, test)= ss_data.randomSplit([.7,.3])
training.describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+
|summary|             label|      temperature|    exhaust_vacuum|  ambient_pressure| relative_humidity|
+-------+------------------+-----------------+------------------+------------------+------------------+
|  count|              6805|             6805|              6805|              6805|              6805|
|   mean| 454.1748082292439|19.73569728141077| 54.47495077149149|1013.2312872887579| 73.25819691403366|
| stddev|16.944753748043656|7.436459195735624|12.690629786503358| 5.924620841614613|14.601364177971531|
|    min|            421.57|             1.81|             25.36|            992.89|             25.56|
|    max|            495.35|            37.11|             80.25|            1033.3|            100.16|
+-------+------------------+-----------------+------------------+------------------+------------------+



In [23]:
#create lnear regression
lr = LinearRegression(maxIter=10)
lr_model=lr.fit(training)

In [24]:
#model coefficient
lr_model.coefficients, lr_model.intercept

(DenseVector([-14.6565, -2.9634, 0.3819, -2.2868]), 452.00505189101585)

In [25]:
summary = lr_model.summary

In [22]:
summary.rootMeanSquaredError, summary.meanAbsoluteError, summary.meanSquaredError, summary.r2

(4.574548473500837, 3.6356092988516138, 20.926493736408833, 0.9271063379739649)

In [26]:
summary.rootMeanSquaredError, summary.meanAbsoluteError, summary.meanSquaredError, summary.r2

(4.574548473500837, 3.6356092988516138, 20.926493736408833, 0.9271063379739649)

In [27]:
from pyspark.ml.evaluation import RegressionEvaluator

In [28]:
ev = RegressionEvaluator(labelCol="label",predictionCol="prediction",metricName="rmse")

In [29]:
pred_df = lr_model.transform(test)

In [31]:
pred_df.select("label","prediction").show(truncate=True)

+------+------------------+
| label|        prediction|
+------+------------------+
|420.26|446.45248689105813|
|425.11| 431.1609697559184|
|425.14|430.89912891100414|
|425.14| 428.2245734336546|
|425.17| 429.4406836541228|
|425.19| 428.2410278708463|
| 425.2| 432.7058849429941|
| 425.5|428.88608248989823|
|425.66|430.82972385530917|
|425.71|430.06806662890443|
|425.89| 428.9860350272594|
|425.89| 426.5471279850598|
|425.91| 430.2312547591599|
| 426.0|432.55995807554467|
|426.15| 430.3510603709572|
|426.18|433.48064538218125|
| 426.2| 432.9522661325599|
|426.25| 431.4233637145474|
|426.29|  432.780990289772|
|426.48| 432.1033158587586|
+------+------------------+
only showing top 20 rows

